[参考1](https://blog.csdn.net/mfq1219/article/details/81945448)

[参考2](https://blog.csdn.net/m0_37822019/article/details/79709617)

ctypes是Python的一个外部库，提供和C语言兼容的数据类型，可以很方便地调用DLL中输出的C接口函数。

## 1. 加载dll和取出函数

```python
from ctypes import *   
dll = cdll.LoadLibrary(dllpath)   #dllpath是字符串
dll = windll.LoadLibrary(dllpath)
```

上面两行使用哪一行,取决于导出函数的调用规范(cdecl或stdcall).也可以使用下面两行代替:

```python
dll = CDLL(dllpath)    #注意和上面大小写的区别
dll = WinDLL(dllpath) 
```

💊注意,这里使用的dll必须和python平台匹配,比如都是32位的或者都是64位的。因为本质上是一个exe加载一个dll，无法跨平台。

加载dll后,可直接得到dll中的导出函数地址.

```python
func = dll.func_name  #func_name 是dll的导出函数
```

有时动态链接库导出c++函数时，并不是有效的Python标识符，例如 "??2@YAPAXI@Z" 。这种情况下，必须使用getattr 获取函数:

```python
func = getattr(cdll.msvcrt,"??2@YAPAXI@Z")
```

在Windows上，有些动态链接库导出函数不是用名字，而是用序号(ordinal)。这时需通过索引获取：

```python
func = cdll.kernel32[1]
```

## 加载C语言标准库

In [1]:
from ctypes import *
from platform import system

your = system()
if your == 'Linux':
    libc =  CDLL("libc.so.6")
elif your == 'Windows':
    libc = cdll.msvcrt
    
    
libc.printf(b"Hello %s\n", b'Trump')

12

## 函数参数和返回值

### 类型对照表

上面只是得到了函数地址,还无法进行函数调用.要进行正确的函数调用,需设置好参数和返回值类型.
ctypes支持的原生数据类型如下:                                    

| ctypes类型     | C 类型                                   | Python 类型                        |
|:------------------ |:-------------------------------------- |:-----------------------------------|
| c_char            | char                                       | 1-character string          |
| c_wchar         | wchar_t                                  | 1-character unicode string |
| c_byte            | char                                       | int/long                   |
| c_ubyte          | unsigned char                        | int/long                   |
| c_bool            | bool                                        | bool                       |
| c_short           | short                                       | int/long                   |
| c_ushort         | unsigned short                       | int/long                   |
| c_int               | int                                           | int/long                   |
| c_uint             | unsigned int                           | int/long                   |
| c_long            | long                                        | int/long                   |
| c_ulong          | unsigned long                        | int/long                   |
| c_longlong     | __int64 or longlong                 | int/long                   |
| c_ulonglong   | unsigned __int64 or unsigned long long | int/long                   |
| c_float            | float                                        | float                                     |
| c_double        | double                                     | float                                     |
| c_longdouble | long double float                      | float                                     |
| c_char_p        | char *                                       | string or None                     |
| c_wchar_p     | wchar_t *                                 | unicode or None                  |
| c_void_p        | void *                                        | int/long or None                  |

### 设置参数类型和返回值类型

- 设置函数的参数类型使用函数的`argtypes`属性,直接赋值为一个ctypes类型的列表或元组。
- 设置函数的返回值类型使用函数的`restype`属性。下面是示例代码

**python中，默认函数返回值是c_int型**，此类型可以不用显示设置函数的`restype`属性，如果是**参数类型是c_int型则需要设置。**

```python
fun.argtypes = (c_int, c_int, c_int, c_void_p) #设置函数参数类型为 int,int,int,void *
fun.restype  = c_float #设置返回值类型为 float
```

**None、整数、字节串和（unicode）字符串是可以作为本地Python对象直接传递给函数调用的。**

- None是作为C的**NULL指针**传递。
- 字节串和字符串作为内存块指针传递(char* 或 wchar_t*)。
- Python整数作为平台相关的C语言int类型传递，**其值会截断到C类型**。

**除了整数、字节串和字符串以外Python类型的参数传递，必须使用ctypes类型做包装。**

在调用函数时，如果使用了错误的参数数量和调用规范时，ctypes尝试保护调用。不幸的是该功能仅在Windows上有用。它通过检查函数返回栈来实现，所以尽管发生了错误，但是函数还是调用了。
这很容易导致当前使用的整个Python环境崩溃，所以必须很小心的使用。

---

## 结构体

除了上述的基本类型，ctypes还支持自定义的结构体和联合体，它们可以出现在函数的参数或返回值中。

### 基本用法

自定义的结构体和联合体必须**继承自ctypes的Structure和Union**，这两个类都在ctypes模块中定义。每一个子类必须定义`_fields_`属性，`_fields_`是一个**二维的tuples列表**，描述类的每个数据成员的字段名和字段类型，这里的字段类型必须是一个ctypes类型，如c_int，或者任何其他的继承ctypes的类型，如Structure, Union, Array, 指针等。

例如有一个简单结构，包含两个整型x和y，可如下初始化一个结构：

In [ ]:
from ctypes import *  
import types  
    
class Point(Structure):  
    _fields_ = [('x', c_int),  
                ('y', c_int)]  
    
p1 = Point(1, 2)  
print(p1.x, p1.y)

1 2


### 嵌套结构体

In [14]:
#可以创建复杂的结构体,嵌套了其它结构体。如下：
class RECT(Structure):
    _fields_ = [("upperleft", Point),
                ("lowerright", Point)]

rc = RECT(p1)
print(rc.upperleft.x, rc.upperleft.y)
print(rc.lowerright.x, rc.lowerright.y) 

1 2
0 0


In [18]:
#嵌套结构体可以通过下面多种方法初始化：
rc2 = RECT(Point(1,2), Point(3,4))
rc3 = RECT((1,2), (3,4))

### 结构体指针

如结构体用于链表操作，即包含指向结构体指针时，若直接定义：

In [1]:
from ctypes import *  
import types  
    
class Test(Structure):  
    _fields_ = [('x', c_int),  
                ('y', c_char),  
                ('next', Test)]   #这一行报错

NameError: name 'Test' is not defined

则python会报错type未定义，如下定义则OK：

In [2]:
from ctypes import *
import types


class Test(Structure):
    pass


Test._fields_ = [('x', c_int), ('y', c_char), ('next', POINTER(Test))]

### 字节对齐和字节顺序

默认情况下结构体和联合的对齐使用C编译器相同的方式。这可以通过类属性`_pack_` 来重载其行为。这必须设置一个正数指定字段的最大对齐。这个功能与MSVC中的 `#pragma pack(n)` 功能一样。

`ctypes`中的结构体和联合使用本地字节序。想要用非本地字节序，可以使用 `BigEndianStructure` 、`LittleEndianStructure` 、 `BigEndianUnion` 、 `LittleEndianUnion` 基类。这些类无法包含指针字段。

### 位域

创建结构与联合体时，可以包含位域字段。**只有整型域才可以使用位字段**，位宽可以在`_fields_`元组的第三个选项中指定：

In [4]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),   #这个字段占16位
                ("second_16",c_int, 16)]  

##  数组

数组就是序列，包含固定数量的相同类型的实例。**推荐的创建数组类型的方式是使用正数和乘号应用到类型:**

In [9]:
from ctypes import *


class POINT(Structure):
    _fields_ = [("x", c_int), 
                ("y", c_int)]


TenPointsArrayType = POINT * 10  #创建一个数组类型,它是10个Point元素组成的数组。
arr = TenPointsArrayType()  #创建一个数组类的对象。
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [10]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]  # 相当于C语言的:  POINT pts[4]


print(len(MyStruct().point_array))  #输出  4

4


In [11]:
TenIntegers = c_int * 10  # 定义一个int[10]的类型（类）

ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  #显式初始化数组(类)
for i in ii:
    print(i)

1
2
3
4
5
6
7
8
9
10


### 高维数组

In [17]:
#高维数组,即数组的数组。
type_int_array_10 = c_int * 10  #先定义一个数组类型
type_int_array_10_10 = type_int_array_10 * 10  #定义数组的数组（即二维数组）
my_array = type_int_array_10_10()  #创建二维数组对象
my_array[1][2] = 3  #使用二维数组

## 指针和引用

有时C函数需要一个指针指向的数据作为参数，或想向指针指向的内存块写输出数据，或者数据太大不适合传递，这时就需要使用指针或者引用。

ctypes中使用`POINTER`和`pointer`表示指针，在使用`POINTER时`需要设置指向的**数据类型**，而`pointer`则直接从**变量**中得到一个特定类型的指针。

ctypes使用 `byref()` 函数表示指针概念，该函数也是直接从变量得到指针指向的数据类型。

对内存块的使用，实际上是区分输入/输出的。如果是该内存块是函数的输入（即函数内部是从内存块读数据），则使用指针，即`POINTER()` 或 `pointer()`。

如果是该内存块是函数的输出（即函数内部是写数据到内存块），则需使用`create_string_buffer()` 函数。

### 指针 POINTER、pointer

例如创建一个类似于C语言的`int *`：

In [18]:
type_p_int = POINTER(c_int)  #创建指针类型,它指向整数
v = c_int(4)  #定义一个整数,值为4.
p_int = type_p_int(v)  #给一个指针变量(p_int)赋值(为变量v的地址).

print(p_int[0])
print(p_int.contents)  #指针实例有一个contents属性，返回这个指针所指向的对象。

4
c_long(4)


In [33]:
type_p_int = POINTER(c_int)  #创建指针类型,它指向整数
v = c_int(4)  #定义一个整数,值为4.
p_int = type_p_int(v)  #给一个指针变量(p_int)赋值(为变量v的地址).
p_int[2] = 3

print(p_int[0], p_int[2])
print(p_int.contents)  #指针实例有一个contents属性，返回这个指针所指向的对象。

4 3
c_long(4)


上面这段代码在C语言里相当于：
```c
typedef int * type_p_int;
int v = 4;
type_p_int p = &v;
printf("%d", p[0]);
printf("%d", *p);
```

也可以不经过声明指针类型这一步，直接从变量得到指针，如下：

In [32]:
v = c_int(4)        #定义一个整数,值为4.
p_int = pointer(v)  #直接得到v的指针，不需创建指针类型（省去类型声明）。

p_int[2] = 3
print(p_int[0], p_int[2])
print(p_int.contents)

4 3
c_long(4)


注意：对指针类型 `c_char_p`，`c_wchar_p`，`c_void_p` 的赋值将会改变其指向的内存区域地址，而不是改变内存块的值**（因为Python字符串是只读的）**。

### 引用：byref()

`ctypes`使用 `byref()` 函数**传递参数引用**。通常使用 `byref()`的地方同样也可用指针函数`pointer()`，但`pointer()`作为参数通常会额外创建一个指针对象，如果并不需要再次使用该指针对象的话，使用 `byref()` 会更快。

## 内存块

各种指针类型（`c_char_p`，`c_wchar_p`，`c_void_p`）指向的内存块实际上都是只读的。如果某个函数需要一个输入内存块保存输出值，不能传递这些指针。我们需要一个可写的内存块，使用`create_string_buffer()` 函数创建。

In [34]:
from ctypes import *

p = create_string_buffer(3)  #创建3字节长的buf，且初始化为0
print(sizeof(p), repr(p.raw))

3 b'\x00\x00\x00'


In [41]:
p = create_string_buffer(b"Hello")  #创建一个字符串(包括结尾的0)的buf
print(sizeof(p))
print(p.raw)
print(p.value)

6
b'Hello\x00'
b'Hello'


In [43]:
p = create_string_buffer(b"Hello", 10)
print(sizeof(p), p.raw)

10 b'Hello\x00\x00\x00\x00\x00'


In [45]:
p.value = b"Hi"  #修改buf内容（这是可变buf）
print(sizeof(p), p.raw) 

10 b'Hi\x00lo\x00\x00\x00\x00\x00'


想要创建包含unicode字符(对应C类型`wchar_t`)的可变内存块，使用`create_unicode_buffer()` 函数。

In [53]:
pw = create_unicode_buffer(3)  #创建一个unicode使用的buf，且初始化为0
print(sizeof(pw), pw.value)  #注意，这里将输出6。

6 


## 类型转换

通常情况下，ctypes会做严格的类型检查。这意味着，如果形参有一个`POINTER(c_int)`指针指向一个函数或者结构体的成员域类型，那么实参只能接受相同类型的实例。
但这个规则也有例外。比如，你可以传递兼容的数组类型来代替指针类型。例如对于`POINTER(c_int)`指针类型来说，可以使用`c_int`数组来代替。

In [56]:
class Bar(Structure):
    _fields_ = [("count", c_int), 
                ("values", POINTER(c_int))]


bar = Bar()
bar.values = (c_int * 3)(1, 2, 3)  #数组和指针的转化
bar.count = 3
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [57]:
bar.values = None  #设置指针为NULL

for i in range(bar.count):
    print(bar.values[i])

ValueError: NULL pointer access

如果一个函数参数显式声明为某种指针类型（例如`POINT(c_int)` 类型），则传递该指针指向的对象类型也是可以的（例如这里可以传递`c_int`），`ctypes`会自动加上`byref()`函数进行类型转换。

在C语言中，你可以通过**强制类型转换**的方法来转换不兼容的类型。ctypes也提供了一个转换函数`cast()` 让你可以使用相同的方式进行类型转换。
`cast()`函数可以将一个ctypes指针（或数组）的实例转换成另外一个不同的指针类型（或数组）。`cast()`函数需要两个参数，第一个是转换前的**指针实例**，
第二个是目标**指针类型。它返回第二个参数类型的实例，并且这个实例与第一个参数共用同一块内存。**

上面定义的`Bar`结构体中，它的`value`域可以支持`POINTER(c_int)`指针或者`c_int`数组，但不支持其他类型，如果需要其它类型，则可使用类型转换。

In [58]:
bar.values = (c_byte * 4)()  #报错，类型不对。需要 int * 或 int 数组。这里是byte数组。

TypeError: incompatible types, c_byte_Array_4 instance instead of LP_c_long instance

In [59]:
bar.values = cast((c_byte * 4)(), POINTER(c_int))  #正确。强制转换,把byte数组转化为int*

In [60]:
print(bar.values[0])

0


## 回调函数

types允许从python回调中创建c回调函数指针。这个常常被称为回调函数。

首先，你必须为回调函数创建一个类，这个类知道调用协议，函数返回值类型，函数接受的参数个数及类型。

`CFUNCTYPE`工厂函数使用普通`cdecl`调用协议来为回调函数**创建类型**。并且，在Windows平台，`WINFUNCTYPE`工厂函数使用`stdcall`调用协议来为回调函数创建类型。
这两个工厂函数在调用时，**参数表都是使用返回值作为第一个参数，而将回调函数所需要的参数作为剩下的参数。**

在这里我将使用一个c标准库里的快排函数作为演示例子，快排是一个借助回调函数进行排序的函数。快排将会用到下面的整型数组：

In [3]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))  #定义回调函数类型


def py_cmp_func(a, b):  #实现回调函数功能
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]


cmp_func = CMPFUNC(py_cmp_func)  #回调函数对象

IntArray5 = c_int * 5
ia = IntArray5(5, 1, 7, 33, 99)
qsort = libc.qsort  #排序函数地址
qsort.restype = None  #排序函数返回值

qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func))  #调用排序函数,传入一个回调函数对象.

for i in ia:
    print(i, end=" ")

#输出结果为： 1 5 7 33 99

py_cmp_func 1 5
py_cmp_func 7 5
py_cmp_func 33 7
py_cmp_func 99 33
py_cmp_func 1 5
py_cmp_func 7 5
py_cmp_func 33 7
py_cmp_func 1 5
py_cmp_func 7 5
py_cmp_func 1 5
1 5 7 33 99 

💊**回调函数的重要提示：**

确保你在C代码的使用生命周期里保持引用CFUNCTYPE对象。ctypes并不会帮你做这样的事情，如果你没有做保证，它们就会被垃圾回收，然后当你调用这个回调函数时将会导致程序崩溃。

## 8.windows类型

Windows API有一些特殊之处，Windows API函数不使用标准C的调用约定（前面已经提到过）。

因此需注意两点：

- LoadLibrary时不能够使用`cdll.LoadLibrary`而使用`windll.LoadLibrary`。
- 在声明回调函数指针类型的时候，不能用`CFUNCTYPE`而是用`WINFUNCTYPE`。

😜Windows API有很多内建类型，ctypes内部都已经定义好了，在子模块`wintypes`下，可以直接使用。

代码举例如下：

In [5]:
from ctypes import *
from ctypes import wintypes

# HWND 等类型已定义好了，可直接使用.
WNDENUMPROC = WINFUNCTYPE(
    wintypes.BOOL,  #定义回调函数类型
    wintypes.HWND,
    wintypes.LPARAM)


def EnumWindowsProc(hwnd, lParam):  #实现回调函数功能
    length = user32.GetWindowTextLengthW(hwnd) + 1
    buffer = create_unicode_buffer(length)
    user32.GetWindowTextW(hwnd, buffer, length)
    print(buffer.value)
    return True


user32 = windll.LoadLibrary('user32.dll')  #加载dll
user32.EnumWindows(WNDENUMPROC(EnumWindowsProc), 0)

## 结构体指针

返回结构体指针的函数. c代码如下:
```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

mystruct * create(){
  mystruct * s = (mystruct *)calloc(1, sizeof(mystruct));
  s->a = 100;
  s->b = 200;
  return s;
}

void destroy(mystruct * s){
  free(s);
}
```
python代码如下：

```py
from ctypes import *


class mystruct(Structure):
    _fields_ = [('a', c_int), 
                ('b', c_int)]


dll = cdll.LoadLibrary(dllpath)
dll.create.restype = POINTER(mystruct)  #设置返回值类型为结构体指针
p = dll.create()  #调用函数
print(p.contents.a, p.contents.b)  #输出 100 200
```

如果返回的是结构体数组指针，同样也是将函数返回类型设置为结构体指针，在接收到值之后，使用时加上下标即可,注意是结构体数组指针，不是结构体指针数组
python代码如下：

```python
from ctypes import *


class mystruct(Structure):
    _fields_ = [('a', c_int), ('b', c_int)]


dll = cdll.LoadLibrary(dllpath)
dll.create.restype = POINTER(mystruct)  #设置返回值类型为结构体指针
p = dll.create()  #调用函数
print(p[i].a, p[i].b)  #注意是结构体数组指针，不是结构体指针数组，不然应该还需要
##如果是向函数内部传入一个结构体数组指针，而函数需要改变此结构体的内存，则可以作如下处理：
```

返回结构体指针参数的函数. c代码如下:
    
```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

 int change(mystruct*stru,int*num){
 
 for(i＝０;i < 3;i++)
  {
   stru[i].a = i;
   stru[i].b = i+1;
  }
  
  int n_num = 2;
  num = &n_num; 
  return 0;
}
```
python代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]
dll = cdll.LoadLibrary(dllpath) 
stru_info= create_string_buffer(sizeof(mystruct) * NUM)
p_rec = POINTER(mystruct)(stru_info)
info_num = c_int()
ret = dll.create(p_rec, byref(info_num)) #调用函数
print(p_rec[i],p_rec[i].b)
```

同理，如果创建int类型的数组内存，则可以用

```python
int_buffer = create_string_buffer(sizeof(c_int) * NUM)
p_int_buffer = POINTER(c_int)(int_buffer)
```

传递参数时直接使用`p_int_buffer`
使用时直接`p_int_buffer[i]`取值即可

如果是向函数内部传入一个结构体数组指针，而函数不需要改变此结构体的内存，则可以作如下处理：
c代码如下:

```c
include <stdlib.h>
typedef struct{
  int a;
  int b;
}mystruct;

 int change(mystruct*stru,int num){
 
 for(i＝０;i < num;i++)
  {
   cout<<stru[i].a<<endl;
   cout<<stru[i].b<<endl;
  }
  
  return 0;
}
```

python代码如下：

```python
from ctypes import *  
class mystruct(Structure):
    _fields_ = [('a', c_int),('b', c_int)]
dll = cdll.LoadLibrary(dllpath) 
info_num = 3
stru_info_list = []#添加一些结构体，此处略去
stru_info_p = (mystruct*info_num)(*stru_info_list)
num = c_int(info_num)
ret = dll.create(byref(strstru_info_p), num) #调用函数
print(p_rec[i],p_rec[i].b)
```

同理，如果只是传递一个数组指针，不需要改变内容的话，假设传第一个int数组：

```python
int_list = []
num = len(int_list)
p_int_list = (c_int*num)(*int_list)
```

### 解析

上述例子中，如果创建int类型的数组内存
假设c函数是
```c
int fun（int*buffer_list）#传递的是int数组的指针
int_buffer = create_string_buffer(sizeof(c_int) * NUM)
```

如果直接传递的参数为`byref（int_buffer）`
即：
```python
ret = dll.fun（byref（int_buffer））
```

那么对于`int_buffer`的使用就需要解析
首先需要取`int_buffer`得二进制内容，即`int_buffer.raw`，然后解析

```python
buffer = int_buffer.raw
buffer_int = (struct.unpack('i', buffer[i * 4 : (i + 1) * 4]))[0]
```